<a href="https://colab.research.google.com/github/leonbenenati/Projeto/blob/main/Churn_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset do Kaggle : Churn Modeling Dataset

Objetivo é prever se o cliente fechou a conta do banco ou não

In [42]:
#importando bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [11]:
dados = pd.read_csv("/content/Churn_Modelling.csv")

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
dados.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [12]:
#transformando as variáveis Gender e Geography em variáveis númericas
dados['Gender'] = dados['Gender'].map({'Female': 1, 'Male': 0})
dados = pd.get_dummies(dados, columns = ['Geography'])

In [13]:
#Padronizando variáveos
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

padro = sc.fit_transform(dados[['CreditScore', 'Age', 'EstimatedSalary','Balance']])
padro= pd.DataFrame(padro, columns = ['escore', 'idade', 'salario','saldo'])
dados = pd.concat([dados, padro], axis = 1)

In [14]:
dados.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,escore,idade,salario,saldo
0,1,15634602,Hargrave,619,1,42,2,0.00,1,1,1,101348.88,1,1,0,0,-0.326221,0.293517,0.021886,-1.225848
1,2,15647311,Hill,608,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1,-0.440036,0.198164,0.216534,0.117350
2,3,15619304,Onio,502,1,42,8,159660.80,3,1,0,113931.57,1,1,0,0,-1.536794,0.293517,0.240687,1.333053
3,4,15701354,Boni,699,1,39,1,0.00,2,0,0,93826.63,0,1,0,0,0.501521,0.007457,-0.108918,-1.225848
4,5,15737888,Mitchell,850,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1,2.063884,0.388871,-0.365276,0.785728


In [22]:
#separando x e y
x = dados[["escore","Gender","idade","Tenure","saldo","NumOfProducts","HasCrCard","IsActiveMember","EstimatedSalary","Geography_Spain","Geography_France"]]
y = dados["Exited"]

In [23]:
#separando dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(x,
                                y,
                                test_size=0.2,
                                random_state=42)

In [18]:
#Configurando a validação cruzada
kf = StratifiedKFold(n_splits = 10)

Melhor KNN

In [29]:
parametros = {'n_neighbors': np.arange(0, 10,1)}

clf_knn = RandomizedSearchCV(KNeighborsClassifier(), param_distributions = parametros,
                                scoring = 'accuracy', cv = kf,
                                n_iter = 50, verbose = 1, n_jobs = -1)
clf_knn.fit(X_train,y_train)

print('\nAcuracia: {:.4f}'.format(clf_knn.best_score_))
print('Melhores parâmetros: {}\n'.format(clf_knn.best_params_))


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 10 is smaller than n_iter=50. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 10 folds for each of 10 candidates, totalling 100 fits

Acuracia: 0.7888
Melhores parâmetros: {'n_neighbors': 8}



[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.5s finished


Arvore de decisão

In [28]:
parametros = {'min_samples_split': np.arange(1,100,2),
                 'min_samples_leaf': np.arange(1,100,2),
                 'splitter': ['best', 'random'],
                 'criterion': ['gini', 'entropy'],
              'max_depth': np.arange(1,100,2)}
clf_arvore_decisao = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions = parametros,
                                scoring = 'accuracy', cv = kf,
                                n_iter = 50, verbose = 1, n_jobs = -1)

clf_arvore_decisao.fit(X_train,y_train)

print('\nAcuracia: {:.4f}'.format(clf_arvore_decisao.best_score_))
print('Melhores parâmetros: {}\n'.format(clf_arvore_decisao.best_params_))

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    4.8s



Acuracia: 0.8516
Melhores parâmetros: {'splitter': 'random', 'min_samples_split': 3, 'min_samples_leaf': 39, 'max_depth': 47, 'criterion': 'gini'}



[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    8.2s finished


Floresta aleatória

In [35]:
parametros = {'max_depth' : np.arange(0,30,1),
                 'n_estimators':np.arange(0,48,1),
                 'max_features': ['sqrt', 'auto', 'log2'],
                 'min_samples_split':np.arange(0,10,1),
                 'min_samples_leaf': np.arange(0,5,1),
                 'bootstrap': [True, False]
                 }

clf_random = RandomizedSearchCV(RandomForestClassifier(), param_distributions = parametros, scoring = 'accuracy', cv = kf,
                                n_iter = 50, verbose = 1, n_jobs = -1)

clf_random.fit(X_train,y_train)

print('\nAcuracia: {:.4f}'.format(clf_random.best_score_))
print('Melhores parâmetros: {}\n'.format(clf_random.best_params_))

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 362 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.0min finished



Acuracia: 0.8614
Melhores parâmetros: {'n_estimators': 47, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 25, 'bootstrap': True}



Combinação de modelos

In [44]:
knn_escolhido = KNeighborsClassifier(n_neighbors = 8)
arvore_decisao= DecisionTreeClassifier(splitter = 'random', min_samples_split= 3, min_samples_leaf= 39, max_depth= 47, criterion= 'gini')
random = RandomForestClassifier(n_estimators = 47, min_samples_split= 6, min_samples_leaf= 2, max_features = 'auto', max_depth= 25, bootstrap= True)

modelos = [('knn', knn_escolhido),('arvore', arvore_decisao),('floresta_aleatória', random)]

modelo = VotingClassifier(modelos, voting = 'hard')

combinacao =cross_val_score(modelo,X_train,y_train, cv = kf, scoring = 'accuracy')

print(combinacao.mean())

0.852875


Previsão

O modelo de melhor acurácia  foi o Floresta aleatória, será utilizado para a previsão 

In [46]:
Previsao = clf_arvore_decisao.predict(X_test)
print(Previsao)

[0 0 0 ... 1 0 0]
